- Прочитайте главы 7-8 из книги "Spark: The Definitive Guide".
- Загрузите датасеты по ссылкам:
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2019
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2020
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2021
    - https://www.kaggle.com/datasets/sveta151/tiktok-popular-songs-2022
- Выполните задания.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import col,lit
spark = SparkSession.builder\
                    .master("local")\
                    .appName('SparkLab3')\
                    .getOrCreate()

# Загружаем наборы данных в один DataFrame.
df0=tiktokData2019 = spark.read.option("inferSchema", "true").option("header", "true").csv("/home/student/Desktop/TikTok_songs_2019.csv")\
.select("track_name", "artist_name", "artist_pop","album","track_pop","danceability","energy","loudness","speechiness", "acousticness", "instrumentalness", "liveness", "valence",lit("2019").alias("year"))
df1=tiktokData2020 = spark.read.option("inferSchema", "true").option("header", "true").csv("/home/student/Desktop/TikTok_songs_2020.csv")\
.select("track_name", "artist_name", "artist_pop","album","track_pop","danceability","energy","loudness","speechiness", "acousticness", "instrumentalness", "liveness", "valence",lit("2020").alias("year"))
df2=tiktokData2021 = spark.read.option("inferSchema", "true").option("header", "true").csv("/home/student/Desktop/TikTok_songs_2021.csv")\
.select("track_name", "artist_name", "artist_pop","album","track_pop","danceability","energy","loudness","speechiness", "acousticness", "instrumentalness", "liveness", "valence",lit("2021").alias("year"))
df3=tiktokData2022 = spark.read.option("inferSchema", "true").option("header", "true").csv("/home/student/Desktop/TikTok_songs_2022.csv")\
.select("track_name", "artist_name", "artist_pop","album","track_pop","danceability","energy","loudness","speechiness", "acousticness", "instrumentalness", "liveness", "valence",lit("2022").alias("year"))
df4 = df0.unionByName(df1, allowMissingColumns = True)
df5 = df4.unionByName(df2, allowMissingColumns = True)
df6 = df5.unionByName(df3, allowMissingColumns = True)   
#df6.show(7)


22/12/30 02:13:41 WARN Utils: Your hostname, student-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/12/30 02:13:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/30 02:13:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


№1:(+) Добавьте столбец year целочисленного типа, который содержит год датасета для каждой строки данных. Выведите самые танцевальные песни из наборов данных вместе с полем year.

|          track_name|artist_name|year|danceability|
|--------------------|-----------|----|------------|
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|

<div style="text-align: center"> only showing top 6 rows </div>

In [11]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import desc
from pyspark.sql.types import IntegerType

df7 = df6\
.sort(desc(col("danceability"))).select("track_name", "artist_name", "year", "danceability").show(6)

+--------------------+-----------+----+------------+
|          track_name|artist_name|year|danceability|
+--------------------+-----------+----+------------+
|  Dancing in My Room|   347aidan|2020|        0.98|
|                 Uno|    Ambjaay|2019|       0.978|
|I'm Looking for M...|   Memetown|2020|       0.977|
|           Hood Baby|       KBFR|2020|       0.974|
|              MICKEY| Lil Yachty|2019|       0.971|
|Muffins In The Fr...|      Tiagz|2020|       0.965|
+--------------------+-----------+----+------------+
only showing top 6 rows



№2:(+) Найдите исполнителей, которым сопоставлено более одного значения популярности. Выведите эти значения в виде списков.

|          artist|popularity|
|----------------|----------|
|            KYLE|  [65, 66]|
|Kero Kero Bonito|  [57, 56]|
|       Kesh Kesh|  [27, 26]|
|    Mahogany Lox|  [38, 37]|
|   Trevor Daniel|  [66, 65]|
|       blackbear|  [79, 80]|

In [12]:
from pyspark.sql.functions import col
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import count

df8 = df6\
.groupBy("artist_name","track_name", "album")\
.agg(collect_set("artist_pop").alias("popularity"))\
.withColumn("a",size("popularity"))\
.where("a > 1")\
.withColumnRenamed("artist_name", "artist")\
.select("artist","popularity").show(6)

+-------------+----------+
|       artist|popularity|
+-------------+----------+
|    Kesh Kesh|  [27, 26]|
| Mahogany Lox|  [37, 38]|
|Trevor Daniel|  [66, 65]|
|    blackbear|  [79, 80]|
+-------------+----------+



№3: Исправьте различающиеся значения популярности исполнителей. Замените их на максимальные значения из полученных в предыдущем задании массивов.

|          track_name|     artist_name|               album|artist_pop|
|--------------------|----------------|--------------------|----------|
|     hot girl bummer|       blackbear|     hot girl bummer|        80|
|Hey Julie! (feat....|            KYLE|Light of Mine (De...|        66|
|             Falling|   Trevor Daniel|             Falling|        66|
|             Falling|   Trevor Daniel|            Nicotine|        66|
|            Flamingo|Kero Kero Bonito|            Flamingo|        57|
|       Take Your Man|    Mahogany Lox|       Take Your Man|        38|
|              Vibin'|       Kesh Kesh|              Vibin'|        27|

<div style="text-align: center"> 968 </div>

In [13]:
from pyspark.sql.functions import array_max
from pyspark.sql.functions import when
import pyspark.sql.functions as F
# tiktokData3 = tiktokData1\


# Выводим различия между новым  и старым датафреймами
substractDf = tiktokData3\
.subtract(tiktokData1)\
.select("track_name", "artist_name", "album", "artist_pop")\
.sort(desc("artist_pop"))

substractDf.show()
tiktokData3.count()

NameError: name 'tiktokData3' is not defined

№4:(+) Сгруппируйте данные по названию трека, названию альбома, имени и популярности певца. Годы датасетов соберите в списки. Популярность треков возьмите из максимального значения. Для столбцов danceability, energy,   speechiness, acousticness, instrumentalness, liveness и valence возьмите среднее значение. Остальные столбцы отбросьте. Сколько получилось строк в сгруппированном DataFrame?

Полученный датафрейм сохраните в новый parquet файл. Это можно сделать методом:

```
df.write.format("parquet").mode("overwrite").save("path to file")
```

Результат при этом сохранится в новую директорию.
Другой способ:

```
df.toPandas().to_parquet("path to file")
```

Результат сохранится в один файл без создания директорий. Для этого способа понадобится библиотека pyarrow.
Установите ее с помощью команд:

```
source ~/Work/bin/activate
pip3 install pyarrow
```

In [14]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import max
from pyspark.sql.functions import avg
from pyspark.sql.functions import size
from pyspark.sql.functions import sort_array

df10 = df6\

df10 = df6.select("track_name", "artist_name", "artist_pop","album","track_pop","danceability","energy","loudness", "year","speechiness","acousticness","instrumentalness","liveness","valence")\
.groupBy("track_name","album" , "artist_name", "artist_pop")\
.agg(collect_list("year").alias("years"),
     max("track_pop").alias("track_pop_max"),
     avg("danceability").alias("danceability_avg"),
     avg("energy").alias("energy_avg"),
     avg("speechiness").alias("speechiness_avg"),
     avg("acousticness").alias("acousticness_avg"),
     avg("instrumentalness").alias("instrumentalness_avg"),
     avg("liveness").alias("avg_liveness_avg"),
     avg("valence").alias("valence_avg"))\



df10.write.format("parquet").mode("overwrite").save("path to file")
df10.count()


853

№5:(+) Отсортируйте песни по количеству вхождений в датасеты и популярности. Выведите их вместе со списками годов наборов данных.

|          track_name|               album|     artist_name|track_pop|               years|
|--------------------|--------------------|----------------|---------|--------------------|
|              Say So|            Hot Pink|        Doja Cat|       80|[2019, 2020, 2021...|
|      Wait a Minute!|        ARDIPITHECUS|          WILLOW|       86|  [2019, 2020, 2022]|
|       Electric Love|            Dopamine|           BØRNS|       82|  [2020, 2021, 2022]|
|           Play Date|Cry Baby (Deluxe ...|Melanie Martinez|       78|  [2020, 2021, 2022]|
|             ROXANNE|             ROXANNE|  Arizona Zervas|       78|  [2019, 2020, 2022]|
|Savage Love (Laxe...|Savage Love (Laxe...|       Jawsh 685|       77|  [2020, 2021, 2022]|

<div style="text-align: center"> only showing top 6 rows </div>

In [15]:
df11 = df6\
.groupBy("artist_name","album","track_name","track_pop")\
.agg(collect_list("year").alias("years"))\
.sort(desc(size("years")), desc("track_pop"))\
.select("track_name", "album" ,"artist_name","track_pop", sort_array("years"))\
.withColumnRenamed("sort_array(years, true)", "years").show(6)


+--------------------+--------------------+----------------+---------+--------------------+
|          track_name|               album|     artist_name|track_pop|               years|
+--------------------+--------------------+----------------+---------+--------------------+
|              Say So|            Hot Pink|        Doja Cat|       80|[2019, 2020, 2021...|
|      Wait a Minute!|        ARDIPITHECUS|          WILLOW|       86|  [2019, 2020, 2022]|
|       Electric Love|            Dopamine|           BØRNS|       82|  [2020, 2021, 2022]|
|           Play Date|Cry Baby (Deluxe ...|Melanie Martinez|       78|  [2020, 2021, 2022]|
|             ROXANNE|             ROXANNE|  Arizona Zervas|       78|  [2019, 2020, 2022]|
|Savage Love (Laxe...|Savage Love (Laxe...|       Jawsh 685|       77|  [2020, 2021, 2022]|
+--------------------+--------------------+----------------+---------+--------------------+
only showing top 6 rows



№6:(+) Найдите все треки с повторяющимися именами. Выполните их сортировку по названию. Выведите названия песен, названия альбомов, имена исполнителей и годы датасетов. Сколько строк в полученной таблице?

|          track_name|               album|   artist_name| years|
|--------------------|--------------------|--------------|------|
|"More Than A Woma...|How Can You Mend ...|      Bee Gees|[2022]|
|"More Than A Woma...|            Greatest|      Bee Gees|[2020]|
|223's (feat. 9lok...|    Melly vs. Melvin|     YNW Melly|[2019]|
|223's (feat. 9lok...|223's (feat. 9lok...|     YNW Melly|[2020]|
|           As It Was|           As It Was|Michael Shynes|[2022]|
|           As It Was|           As It Was|  Harry Styles|[2022]|

<div style="text-align: center"> only showing top 6 rows </div>

In [16]:
from pyspark.sql.functions import arrays_zip, col, explode
df12 = df6\
.groupBy("track_name")\
.agg(collect_list("track_name").alias("track_name_n"),
     collect_list("album").alias("album_n"),
     collect_list("artist_name").alias("artist_name_n"),
     collect_list("year").alias("years"))\
.where(count("track_name") > 1)\
.withColumn("tmp", arrays_zip("track_name_n", "album_n" ,"artist_name_n","years"))\
.withColumn("tmp", explode("tmp"))\
.select(col("tmp.track_name_n"), col("tmp.album_n"), col("tmp.artist_name_n"), col("tmp.years"))\

df12.show(6)
df12.count()



+--------------------+--------------------+-------------+-----+
|        track_name_n|             album_n|artist_name_n|years|
+--------------------+--------------------+-------------+-----+
|"More Than A Woma...|            Greatest|     Bee Gees| 2020|
|"More Than A Woma...|How Can You Mend ...|     Bee Gees| 2022|
|223's (feat. 9lok...|223's (feat. 9lok...|    YNW Melly| 2020|
|223's (feat. 9lok...|    Melly vs. Melvin|    YNW Melly| 2019|
|               34+35|           Positions|Ariana Grande| 2021|
|               34+35|           Positions|Ariana Grande| 2022|
+--------------------+--------------------+-------------+-----+
only showing top 6 rows



308